In [44]:
import requests, json, datetime, sqlite3, psycopg2

from IPython.display import JSON, display

In [15]:
api_key = "4eb35303954a7829e790a2f4142eedaa"

In [16]:
city = "Lagos"

In [17]:
url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"

In [18]:
response = requests.get(url)
print(response)

<Response [200]>


In [19]:
data = response.json()
print(data)

{'coord': {'lon': 3.75, 'lat': 6.5833}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 301.87, 'feels_like': 306.93, 'temp_min': 301.87, 'temp_max': 301.87, 'pressure': 1008, 'humidity': 79, 'sea_level': 1008, 'grnd_level': 1008}, 'visibility': 10000, 'wind': {'speed': 2.72, 'deg': 213, 'gust': 6.34}, 'clouds': {'all': 100}, 'dt': 1741457210, 'sys': {'country': 'NG', 'sunrise': 1741413287, 'sunset': 1741456621}, 'timezone': 3600, 'id': 2332453, 'name': 'Lagos', 'cod': 200}


In [22]:
# display(JSON(data))

## Transformation

In [33]:
city_name=data['name']
temperature_celsius=data['main']['temp']
humidity=data['main']['humidity']
wind_speed_kmh=(data['wind']['speed'] * 60 * 60) / 1000
weather_description=data['weather'][0]['description']
timestamp=datetime.datetime.fromtimestamp(data['dt'])

## Load data (using sqlite)

In [34]:
# Create connection to sqlite database
conn = sqlite3.connect(':memory:')

In [35]:
# Add cursor to the connection
cursor = conn.cursor()

In [36]:
cursor.execute('''
    CREATE TABLE weather_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        city TEXT,
        temperature_celsius REAL,
        humidity INTEGER,
        wind_speed_kmh REAL,
        weather_description TEXT,
        timestamp TEXT
    )
''')

conn.commit()

In [42]:
cursor.execute('''
    INSERT INTO weather_data (city, temperature_celsius, humidity, wind_speed_kmh, weather_description, timestamp)
    VALUES (?, ?, ?, ?, ?, ?)
''', (city_name, temperature_celsius, humidity, wind_speed_kmh, weather_description, timestamp))

conn.commit()

In [48]:
cursor.execute('''
    SELECT * FROM weather_data
''')

rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

(1, 'Lagos', 301.87, 79, 9.792000000000002, 'overcast clouds', datetime.datetime(2025, 3, 8, 19, 6, 50))


## Saving to a PostgreSQL

In [55]:
conn = psycopg2.connect(
    host="localhost",
    database="weatherDB",
    user="postgres",
    password="password"
)

cursor = conn.cursor()

In [51]:
insert_query = '''
    INSERT INTO weather_data (city, temperature_celsius, humidity, wind_speed_kmh, weather_description, timestamp)
    VALUES (%s, %s, %s, %s, %s, %s)
'''

cursor.execute(insert_query, (city_name, temperature_celsius, humidity, wind_speed_kmh, weather_description, timestamp))
conn.commit()

In [57]:
cursor.execute('''
    SELECT * FROM weather_data
''')

rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

(1, 'Lagos', 301.87, 79, 9.792000000000002, 'overcast clouds', datetime.datetime(2025, 3, 8, 19, 6, 50))
